In [1]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# 重み付き平均MAPE表の作成@CG

In [2]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [3]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

# 重み付き平均MAPE表の作成@MG

In [4]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

mg_input_list_problem_size: list[int] = [32, 64, 128, 256]
mg_input_list_nit: list[int] = [4, 10, 20, 35]

mg_target_list_problem_size: list[int] = [512]
mg_target_list_nit: list[int] = [50]

# input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=input_list_process,
#     list_na=cg_input_list_na,
#     list_nonzer=cg_input_list_nonzer,
#     list_niter=cg_input_list_niter,
#     list_shift=cg_input_list_shift,
#     csvDir=csvDirPath,
# )
# input_rawDF_cg = input_rawDF_cg.rename(columns={"Name":"functionName"})

input_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=input_list_process,
    list_problem_size=mg_input_list_problem_size,
    list_nit=mg_input_list_nit,
    csvDir=csvDirPath,
)
input_rawDF_mg = input_rawDF_mg.rename(columns={"Name": "functionName"})


# target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=target_list_process,
#     list_na=cg_target_list_na,
#     list_nonzer=cg_target_list_nonzer,
#     list_niter=cg_target_list_niter,
#     list_shift=cg_target_list_shift,
#     csvDir=csvDirPath,
# )
# target_rawDF_cg = target_rawDF_cg.rename(columns={"Name":"functionName"})

target_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=target_list_process,
    list_problem_size=mg_target_list_problem_size,
    list_nit=mg_target_list_nit,
    csvDir=csvDirPath,
)
target_rawDF_mg = target_rawDF_mg.rename(columns={"Name": "functionName"})

mg_list_exp: list[str] = ["process", "problem_size", "nit"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_mg["functionName"])))

In [5]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_mg[
        input_rawDF_mg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_mg[
        target_rawDF_mg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=mg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")
    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[mg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_mg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [6]:
retNum_mg

38.136058981254756

In [7]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [8]:
date: str = "2022年8月26日"


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

targetFunctionNames: list[str] = [
    "VECSET",
    "MPI_Wait()",
    "MPI_Send()",
    "SPRNVC",
    "RANDLC",
    "MPI_Irecv()",
    "ICNVRT",
]

function_names: list[str] = list(set(input_rawDF_cg["Name"].tolist()))

input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})


list_modelName_dict: dict[str, str] = {
    "modelIp": "線形",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelLinearSumOf2elementCombination": "全変数の組合せ積の総和",
    "modelLinearSumOfElementCombinations": "全変数の組合せ積の線形和",
    "modelLinearSumOf2elementCombinationWithSquared": "2要素の積の線形和（2乗含む）",
    "modelLinearSumOf2elementCombinationWithCubed": "2要素の積の線形和（3乗含む）",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses": "問題サイズの変数の総乗とプロセス数とをかける",
    "modelInfiniteProductOfProblemSizeDividedByProcesses": "問題サイズの変数の総乗をプロセス数で割る",
    "modelSquareRootOfProcess": "プロセス数の平方根",
    "modelSquareRootTimesOtherElems": "プロセス数の平方根と他説明変数の積",
    "modelObeyOneParameter": "一つのパラメータに依存する",
    "modelLin": "線形モデル"
    # "modelBasicTree",
}

/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [9]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_before = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_before

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,functionName,最低値,最適モデル
0,2.550709e+01,63.602611,43.918416,68.457739,68.707184,66.810509,66.800152,63.602603,43.918418,2.550710e+01,43.918426,2.550709e+01,63.602600,13.725285,1.353524e+01,26.548026,36.391518,ICNVRT,1.353524e+01,modelLinearSumOfElementCombinations
1,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,24.184348,3.033609e-14,40.738362,14.024686,2.359154e-12,19.369661,25.924923,SPRNVC,3.033609e-14,modelLogAndLin
2,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Send(),1.193837e+01,modelLinearSumOfElementCombinations
3,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,24.184348,3.033609e-14,40.738362,14.024686,2.359154e-12,19.369661,25.924923,VECSET,3.033609e-14,modelLogAndLin
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Irecv(),1.193837e+01,modelLinearSumOfElementCombinations
5,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Wait(),1.193837e+01,modelLinearSumOfElementCombinations
6,2.550716e+01,63.602569,43.918421,68.457716,68.703722,66.810487,66.800130,63.602561,43.918423,2.550717e+01,43.918430,2.550716e+01,63.602559,13.725277,1.353524e+01,26.547929,36.391398,RANDLC,1.353524e+01,modelLinearSumOfElementCombinations


In [10]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [11]:
print(
    resultDF_before.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正前）", siunitx=True)
)

print(f"retNum_cg_before = {retNum_cg_before}")

\begin{table}
\caption{（修正前）}
\label{2022年8月26日_resultDF_before}
\begin{tabular}{llSl}
{} & {関数名} & {重み付きMAPE} & {最適モデル} \\
0 & ICNVRT & 13.535235 & 全変数の組合せ積の線形和 \\
4 & MPI_Irecv() & 11.938374 & 全変数の組合せ積の線形和 \\
2 & MPI_Send() & 11.938374 & 全変数の組合せ積の線形和 \\
5 & MPI_Wait() & 11.938374 & 全変数の組合せ積の線形和 \\
6 & RANDLC & 13.535239 & 全変数の組合せ積の線形和 \\
1 & SPRNVC & 0.000000 & 対数＆線形 \\
3 & VECSET & 0.000000 & 対数＆線形 \\
\end{tabular}
\end{table}

retNum_cg_before = 7.827486063247428


In [12]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_after

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,...,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,modelSquareRootOfProcess,modelSquareRootTimesOtherElems,modelObeyOneParameter,functionName,最低値,最適モデル
0,2.550709e+01,63.602611,43.918416,68.457739,68.707184,66.810509,66.800152,63.602603,43.918418,2.550710e+01,...,13.725285,1.353524e+01,26.548026,36.391518,2.550709e+01,110.789284,25.507090,ICNVRT,13.535235,modelLinearSumOfElementCombinations
1,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,...,14.024686,2.359154e-12,19.369661,25.924923,3.326853e-13,79.407704,0.000000,SPRNVC,0.000000,modelObeyOneParameter
2,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Send(),11.938374,modelLinearSumOfElementCombinations
3,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,...,14.024686,2.359154e-12,19.369661,25.924923,3.326853e-13,79.407704,0.000000,VECSET,0.000000,modelObeyOneParameter
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Irecv(),11.938374,modelLinearSumOfElementCombinations
5,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Wait(),11.938374,modelLinearSumOfElementCombinations
6,2.550716e+01,63.602569,43.918421,68.457716,68.703722,66.810487,66.800130,63.602561,43.918423,2.550717e+01,...,13.725277,1.353524e+01,26.547929,36.391398,2.550716e+01,110.789118,25.507161,RANDLC,13.535239,modelLinearSumOfElementCombinations


In [13]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [14]:
print(
    resultDF_after.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正後）", siunitx=True)
)

print(f"retNum_cg_after = {retNum_cg_after}")

\begin{table}
\caption{（修正後）}
\label{2022年8月26日_resultDF_before}
\begin{tabular}{llSl}
{} & {関数名} & {重み付きMAPE} & {最適モデル} \\
0 & ICNVRT & 13.535235 & 全変数の組合せ積の線形和 \\
4 & MPI_Irecv() & 11.938374 & 全変数の組合せ積の線形和 \\
2 & MPI_Send() & 11.938374 & 全変数の組合せ積の線形和 \\
5 & MPI_Wait() & 11.938374 & 全変数の組合せ積の線形和 \\
6 & RANDLC & 13.535239 & 全変数の組合せ積の線形和 \\
1 & SPRNVC & 0.000000 & 一つのパラメータに依存する \\
3 & VECSET & 0.000000 & 一つのパラメータに依存する \\
\end{tabular}
\end{table}

retNum_cg_after = 7.827486063247428


In [15]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# 重み付き平均MAPEの算出@CG

In [16]:
date: str = "2022年8月26日"

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [17]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    # "modelLinearSumOf2elementCombinationWithSquared",
    # "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_before={retNum_cg_before}")

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

retNum_cg_before=7.827486063247428


In [18]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_after={retNum_cg_after}")

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

retNum_cg_after=7.827486063247428


In [19]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [20]:
date: str = "2022年9月2日"

target_list_process: list[int] = [256]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_niter=cg_target_list_niter,
    list_nonzer=cg_target_list_nonzer,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,na,nonzer,niter,shift
0,100.0,0.00616,1:44.607,1.000000e+00,1.000000e+00,.TAU_application,256,1500000,21,100,200
1,100.0,35,1:44.607,1.000000e+00,1.334000e+03,CG,256,1500000,21,100,200
2,67.1,"40,154",1:10.141,1.010000e+02,1.024140e+05,CONJ_GRAD,256,1500000,21,100,200
3,31.9,642,"33,350",1.000000e+00,3.000000e+06,MAKEA,256,1500000,21,100,200
4,30.9,"15,810","32,279",1.500000e+06,1.321420e+08,SPRNVC,256,1500000,21,100,200
5,16.7,"17,463","17,463",3.454200e+04,0.000000e+00,MPI_Wait(),256,1500000,21,100,200
6,11.7,"12,284","12,284",3.454200e+04,0.000000e+00,MPI_Send(),256,1500000,21,100,200
7,10.5,"10,999","10,999",8.809450e+07,0.000000e+00,RANDLC,256,1500000,21,100,200
8,5.2,"5,469","5,469",4.404720e+07,0.000000e+00,ICNVRT,256,1500000,21,100,200
9,0.5,0.0388,559,1.000000e+00,3.003910e+00,INITIALIZE_MPI,256,1500000,21,100,200


In [21]:
# cg_target_list_na: list[int] = [1500000]
# cg_target_list_nonzer: list[int] = [21]
# cg_target_list_niter: list[int] = [100]
# cg_target_list_shift: list[int] = [200]

get_execTime(
    benchmarkName="cg",
    condition={"process": 256, "na": 1500000, "nonzer": 21, "niter": 100, "shift": 200},
    csvDir=csvDirPath,
)

104.607

In [22]:
test_convertPprofTime()

In [23]:
target_rawDF_cg["Exclusive"]

_before_converted = list(target_rawDF_cg["Exclusive"])

_after_converted = map(convertPprofTime, _before_converted)

_after_converted

target_rawDF_cg["converted_Exclusive"] = list(_after_converted)

target_rawDF_cg

,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,na,nonzer,niter,shift,converted_Exclusive
0,100.0,0.00616,1:44.607,1.000000e+00,1.000000e+00,.TAU_application,256,1500000,21,100,200,6.160000e-06
1,100.0,35,1:44.607,1.000000e+00,1.334000e+03,CG,256,1500000,21,100,200,3.500000e-02
2,67.1,"40,154",1:10.141,1.010000e+02,1.024140e+05,CONJ_GRAD,256,1500000,21,100,200,4.015400e+01
3,31.9,642,"33,350",1.000000e+00,3.000000e+06,MAKEA,256,1500000,21,100,200,6.420000e-01
4,30.9,"15,810","32,279",1.500000e+06,1.321420e+08,SPRNVC,256,1500000,21,100,200,1.581000e+01
5,16.7,"17,463","17,463",3.454200e+04,0.000000e+00,MPI_Wait(),256,1500000,21,100,200,1.746300e+01
6,11.7,"12,284","12,284",3.454200e+04,0.000000e+00,MPI_Send(),256,1500000,21,100,200,1.228400e+01
7,10.5,"10,999","10,999",8.809450e+07,0.000000e+00,RANDLC,256,1500000,21,100,200,1.099900e+01
8,5.2,"5,469","5,469",4.404720e+07,0.000000e+00,ICNVRT,256,1500000,21,100,200,5.469000e+00
9,0.5,0.0388,559,1.000000e+00,3.003910e+00,INITIALIZE_MPI,256,1500000,21,100,200,3.880000e-05


In [24]:
sum(target_rawDF_cg["converted_Exclusive"])

104.60017214400003

In [25]:
test_ret_relativeCost()

In [26]:
import csv


benchmarkName: str = "cg"
conditions: dict[str, list[int]] = {
    "process": [2, 4],
    "na": [14000, 75000],
    "nonzer": [11, 21],
    "niter": [90, 100],
    "shift": [60, 80],
}
ret = gen_ExtraPinputData(
    benchmarkName=benchmarkName, conditions=conditions, csvDir=csvDirPath
)
print(ret)

PARAMETER na
PARAMETER niter
PARAMETER nonzer
PARAMETER process
PARAMETER shift

POINTS ( 14000 90 11 2 60 )
POINTS ( 14000 90 11 2 80 )
POINTS ( 14000 100 11 2 60 )
POINTS ( 14000 100 11 2 80 )
POINTS ( 14000 90 21 2 60 )
POINTS ( 14000 90 21 2 80 )
POINTS ( 14000 100 21 2 60 )
POINTS ( 14000 100 21 2 80 )
POINTS ( 75000 90 11 2 60 )
POINTS ( 75000 90 11 2 80 )
POINTS ( 75000 100 11 2 60 )
POINTS ( 75000 100 11 2 80 )
POINTS ( 75000 90 21 2 60 )
POINTS ( 75000 90 21 2 80 )
POINTS ( 75000 100 21 2 60 )
POINTS ( 75000 100 21 2 80 )
POINTS ( 14000 90 11 4 60 )
POINTS ( 14000 90 11 4 80 )
POINTS ( 14000 100 11 4 60 )
POINTS ( 14000 100 11 4 80 )
POINTS ( 14000 90 21 4 60 )
POINTS ( 14000 90 21 4 80 )
POINTS ( 14000 100 21 4 60 )
POINTS ( 14000 100 21 4 80 )
POINTS ( 75000 90 11 4 60 )
POINTS ( 75000 90 11 4 80 )
POINTS ( 75000 100 11 4 60 )
POINTS ( 75000 100 11 4 80 )
POINTS ( 75000 90 21 4 60 )
POINTS ( 75000 90 21 4 80 )
POINTS ( 75000 100 21 4 60 )
POINTS ( 75000 100 21 4 80 )

REGION

In [27]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [28]:
# 元データによる結果を取得する
# ✅1. Extra-Pに読み込ますデータの取得
# 2. 取得できた数式から予測


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

str_ExtraPinputData: str = gen_ExtraPinputData(
    benchmarkName="cg",
    conditions={
        "process": input_list_process,
        "na": cg_input_list_na,
        "nonzer": cg_input_list_nonzer,
        "niter": cg_input_list_niter,
        "shift": cg_input_list_shift,
    },
)

filePath = "./extra-p_docker/share/input_cg.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_308/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [29]:
%sx pwd
%sx which extrap
%ls
%sx extrap --text /workspace/extra-p_docker/share/input_cg.txt

Dockerfile*    csv_files/           lib/                usedNote.ipynb*
Pipfile*       docker-compose.yml*  root_jupyter/       usedNote2.ipynb*
Pipfile.lock*  docs/                testing_pri.ipynb*  usedNote3.ipynb*
README.md*     draft.ipynb          testing_pub.ipynb*  usedNote4.ipynb*
app/           extra-p_docker/      txt_files/          used_scripts/


['',
 'Loading file: |          | [00:00<?]ERROR: The given file path is not valid.',
 '',
 'Loading file: |          | [00:00<?]']

In [30]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

str_ExtraPinputData: str = gen_ExtraPinputData(
    benchmarkName="lulesh",
    conditions={
        "process": train_lulesh_processes,
        "iteration": train_lulesh_iterations,
        "size": train_lulesh_sizes,
    },
    version=2,
)

filePath = "./extra-p_docker/share/input_lulesh.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

In [31]:
%sx extrap --text /workspace/extra-p_docker/share/input_lulesh.txt

['',
 'Loading file: |          | [00:00<?]ERROR: The given file path is not valid.',
 '',
 'Loading file: |          | [00:00<?]']

In [32]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# TODO

1. ✅CG, MGで時間をExtra-Pから取得できることを確認
2. ✅時間の単位を「秒」にする
    * 列「Inclusive total msec」に一部存在している”：”で区切られた値の単位は、msec ではなく分と秒を区切る”：”
3. ✅相対コストの算出関数の実装
4. ✅Extra-P のDocker環境の構築
4. Extra-P による予測との組み合わせを行う
    * なにをどう組み合わせるのかがわかっていないので、それは確かめる
    * 元データ, Extra-P単体で予測したデータ, <何か> で結果を作成

# 予測結果に必要なもの

* ✅元データ（予測環境の生データ）
* ✅Extra-P単体で予測したデータ
* Extra-Pで各関数の実行時間を予測し、それにコール回数を掛けた値で予測したデータ
    * ✅コール回数で実行時間を割る
    * ✅Extra-Pでモデルを作成 <- コール回数で実行時間を割る
    * ✅👆を利用して予測
    * ✅本プログラムでモデルを作成 <- コール回数で実行時間を割る
    * ✅👆を利用して予測
    * ✅「コール回数 * 実行時間」を計算して予測

上記のセルよりモデルは次式

$$ 2.5521930556315375 + 0.013737041505459383 * iteration + 3.41165137739477 * 10^7 * iteration * size^3 * \log_2 size $$

In [4]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

In [5]:
trainDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
trainDF_lulesh["InclusivePerCall"] = -1
trainDF_lulesh = trainDF_lulesh.reset_index()
for i, sr in trainDF_lulesh.iterrows():
    trainDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(
        sr["Inclusive"]
    ) / float(sr["#Call"])
trainDF_lulesh = trainDF_lulesh.sort_values("#Call")
# Inclusive列の整形
_before_converted: list[float] = list(trainDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
trainDF_lulesh["Inclusive"] = list(_after_converted)

trainDF_lulesh

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size,InclusivePerCall
4799,31,0.0,0.000283,2.830000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,128,64,9.706906e-05
4671,31,0.0,0.000283,2.830000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,128,16,9.706906e-05
4031,31,0.0,0.000245,2.450000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,8,16,8.403505e-05
4063,31,0.0,0.000259,2.590000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,8,24,8.883706e-05
4095,31,0.0,0.000245,2.450000e-07,2.915450e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,343,8,32,8.403505e-05
...,...,...,...,...,...,...,...,...,...,...,...
772,4,9.3,"4,943",4.943000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,8,128,64,1.461708e-07
3176,8,5.6,"7,406",7.406000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,125,128,64,2.190049e-07
3972,4,7.6,"4,979",4.979000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,216,128,64,1.472354e-07
4774,6,7.5,"4,982",4.982000e+00,3.381660e+07,0.0,Real_t_CalcElemVolume(const,343,128,64,1.473241e-07


In [6]:
testDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
testDF_lulesh["InclusivePerCall"] = -1
testDF_lulesh = testDF_lulesh.reset_index()
for i, sr in testDF_lulesh.iterrows():
    testDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(sr["Inclusive"]) / float(
        sr["#Call"]
    )
testDF_lulesh = testDF_lulesh.sort_values("#Call")

# Inclusive列の整形
_before_converted: list[float] = list(testDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
testDF_lulesh["Inclusive"] = list(_after_converted)

testDF_lulesh

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size,InclusivePerCall
31,31,0.0,0.000256,2.560000e-07,1.953120e-03,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,512,256,128,1.310723e-04
28,28,0.0,0.00152,3.580000e-06,1.000000e+00,2.0,void_ParseCommandLineOptions(int_char_**_Int_t...,512,256,128,3.580000e-06
27,27,0.0,0.0122,1.220000e-05,1.000000e+00,0.0,void_InitMeshDecomp(Int_t_Int_t_Int_t,512,256,128,1.220000e-05
26,26,0.0,0.0299,2.990000e-05,1.000000e+00,0.0,void_Domain::SetupSymmetryPlanes(Int_t),512,256,128,2.990000e-05
25,25,0.0,1,1.000000e-03,1.000000e+00,0.0,void_Domain::~Domain(),512,256,128,1.000000e-03
23,23,0.0,4,4.000000e-03,1.000000e+00,0.0,void_Domain::SetupCommBuffers(Int_t),512,256,128,4.000000e-03
22,22,0.0,7,7.000000e-03,1.000000e+00,0.0,void_Domain::SetupBoundaryConditions(Int_t),512,256,128,7.000000e-03
20,20,0.0,19,1.900000e-02,1.000000e+00,0.0,void_Domain::SetupElementConnectivities(Int_t),512,256,128,1.900000e-02
19,19,0.0,22,2.200000e-02,1.000000e+00,1.0,void_Domain::CreateRegionIndexSets(Int_t_Int_t),512,256,128,2.200000e-02
17,17,0.0,39,3.900000e-02,1.000000e+00,0.0,MPI_Barrier(),512,256,128,3.900000e-02


In [7]:
# ✅関数ごとにfor文を回す
#   * 今回は試験的に１種類の関数で実施
# ✅ExtraP向けの入力データを作成（総実行時間）
# ✅ExtraP向けの入力データを作成（１回実行あたり）
# ✅ExtraPでモデル構築（総実行時間）
# ✅ExtraPでモデル構築（１回実行あたり）
# ✅ExtraPで時間を予測（総実行時間）
# ✅ExtraPで時間を予測（１回実行あたり）
# 関数コール回数予測のためのモデルを構築
# 関数コール回数を予測
# 予測された実行時間と予測されたコール回数をまとめる

functionNames: list[str] = sorted(list(set(trainDF_lulesh["Name"])))
for functionName in functionNames:
    pass

functionName = "Real_t_CalcElemVolume(const"
trainDF_perFunc: pd.DataFrame = trainDF_lulesh[trainDF_lulesh["Name"] == functionName]
testDF_perFunc: pd.DataFrame = testDF_lulesh[testDF_lulesh["Name"] == functionName]

expVar: list[str] = ["process", "iteration", "size"]
resVar: str

In [8]:
# 総実行時間

In [9]:
resVar = "Inclusive"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep -v Loading | grep -v Measurement

['',
 'Loading file: |          | [00:00<?]',
 'Loading file: 100%|██████████| [00:00<00:00, Creating calltree]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 '',
 'Generating models: |          | [00:00<?]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Callpath: reg',
 '\tMetric: time',
 '\t\tModel: 0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)',
 '\t\tRSS: 6.40E+00',
 '\t\tAdjusted R^2: 9.69E-01',
 '']

$$ 総実行時間 =  0.008938888890764687 + 0.0003391531590741526 * iteration + 2.645514630961448 * 10^{-8} * iteration * size^3 * \log_2{size} $$

In [10]:
# 上式より予測対象環境での総実行時間は
time_all = (
    0.008938888890764687
    + 0.0003391531590741526 * test_lulesh_iterations[0]
    + 2.645514630961448e-08
    * test_lulesh_iterations[0]
    * test_lulesh_sizes[0] ** (3)
    * np.log2(test_lulesh_sizes[0])
)
print(time_all)

99.51675178196686


In [11]:
# 1回実行あたり

In [12]:
resVar = "InclusivePerCall"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep -v Loading | grep -v Measurement

['',
 'Loading file: |          | [00:00<?]',
 'Loading file: 100%|██████████| [00:00<00:00, Creating calltree]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 '',
 'Generating models: |          | [00:00<?]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Callpath: reg',
 '\tMetric: time',
 '\t\tModel: 1.5456768003451272e-07',
 '\t\tRSS: 4.66E-14',
 '\t\tAdjusted R^2: 1.00E+00',
 '']

$$ 一回当たりの実行時間 = 1.5456768003451272 * 10^{-7} $$

In [13]:
# 上式より、予測対象環境での1回あたりの実行時間は
time_perCall = 1.5456768003451272 * 10 ** (-7)
print(time_perCall)

1.5456768003451272e-07


In [14]:
# 関数コール回数予測のためのモデルを構築

resVar = "#Call"

result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

trainDF_perFunc = trainDF_perFunc.rename({"Name": "functionName"}, axis="columns")
testDF_perFunc = testDF_perFunc.rename({"Name": "functionName"}, axis="columns")

models = Models(
    inputDF=trainDF_perFunc,
    expVarColNames=expVar,
    resVarColNames=[resVar],
    targetDF=None,
    modelNames=list_modelName,
)

models.setUpDataBeforeCalcLr()
models.calcLr()
models.calcMAPE()


dictCalcedMAPE = models.returnCalculatedMAPE()
for key in dictCalcedMAPE.keys():
    dictCalcedMAPE[key] = float(dictCalcedMAPE[key])
dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
dict_for_series["functionName"] = functionName

series: pd.Series = pd.Series(dict_for_series)
result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)


resultDF_after

,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,...,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,modelSquareRootOfProcess,modelSquareRootTimesOtherElems,modelObeyOneParameter,functionName,最低値,最適モデル
0,1127.007556,1256.690139,1340.469621,108.958047,110.715312,103.503054,100.431976,1256.690139,1340.469621,1127.007556,...,465.142901,178.22039,179.365505,10.878828,1127.007556,950.615455,1127.007556,Real_t_CalcElemVolume(const,10.878828,modelLinearSumOf2elementCombinationWithCubed


In [15]:
bestModelDict: dict = return_bestModelObject(
    inputDF=trainDF_perFunc,
    list_expVar=expVar,
    list_resVar=[resVar],
    list_modelName=list_modelName,
)

bestModel = bestModelDict["object"]

predicted = float(np.array(bestModel.predict(inputDF=testDF_perFunc[expVar])))
_call: float = float(testDF_perFunc.iloc[0][resVar])
_MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
_series: pd.Series = pd.Series(
    {
        "functionName": functionName,
        "call": _call,
        "MAPE": _MAPE,
        "predicted_call": predicted,
    }
)

call = _call

_series

functionName      Real_t_CalcElemVolume(const
call                              538968000.0
MAPE                                 0.600286
predicted_call               542203350.195741
dtype: object

In [16]:
testDF_perFunc

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,functionName,process,iteration,size,InclusivePerCall
4,4,6.9,1:19.552,79.552,538968000.0,0.0,Real_t_CalcElemVolume(const,512,256,128,1.476006e-07


In [17]:
real_time = testDF_lulesh.loc[0]["Inclusive"]

print(
    f"real_time = {real_time}\ntime_all = {time_all}\ntime_perCall * call= {time_perCall * call}\n"
)

real_time = 1149.925
time_all = 99.51675178196686
time_perCall * call= 83.30703337284125



# Extra-P の出力の制御に関する実験

In [18]:
from sympy import sympify, pprint, symbols, log
from sympy.parsing.sympy_parser import parse_expr

eq1: str = """0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)"""
eq2: str = """1.5456768003451272e-07"""

In [19]:
dict_symbols = {}
for elem in expVar:
    dict_symbols[elem] = symbols(elem, real=True)
dict_symbols

{'process': process, 'iteration': iteration, 'size': size}

In [20]:
loaded_eq1 = sympify(eq1, locals=dict_symbols)

loaded_eq2 = sympify(eq2, locals=dict_symbols)

In [21]:
print(type(loaded_eq1))
loaded_eq1

<class 'sympy.core.add.Add'>


2.645514630961448e-8*iteration*size**3*log2(size) + 0.0003391531590741526*iteration + 0.008938888890764687

In [22]:
ans = loaded_eq1.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
ans

14.20299852633616*log2(128) + 0.09576209761374775

In [23]:
ans.evalf()

14.2029985263362*log2(128) + 0.0957620976137478

In [24]:
print(type(ans))

<class 'sympy.core.add.Add'>


In [25]:
eq = sympify("log2(size) + iteration + size", locals=dict_symbols)
ans_eq = eq.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
print(type(ans_eq))
print(type(ans_eq.evalf()))
print(ans_eq.evalf())

<class 'sympy.core.add.Add'>
<class 'sympy.core.add.Add'>
log2(128) + 384.0


In [26]:
eq = sympify("ln(size)/ln(2) + iteration + size", locals=dict_symbols)
ans_eq = eq.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
print(type(ans_eq))
print(type(ans_eq.evalf()))
print(ans_eq.evalf())

<class 'sympy.core.add.Add'>
<class 'sympy.core.numbers.Float'>
391.000000000000


In [27]:
eq = sympify("ln(2) + iteration + size", locals=dict_symbols)
ans_eq = eq.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
print(type(ans_eq))
print(type(ans_eq.evalf()))
print(ans_eq.evalf())

<class 'sympy.core.add.Add'>
<class 'sympy.core.numbers.Float'>
384.693147180560


In [28]:
type(ans_eq.evalf())

sympy.core.numbers.Float

In [29]:
loaded_eq2

1.5456768003451272e-7

In [30]:
import re

inputStr: str = """0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)"""

pattern = re.compile(r"log(?P<base>[0-9])")

result = pattern.sub(r"1/ln(\g<base>)ln", inputStr)
print(result)

0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * 1/ln(2)ln(size)^(1)


In [31]:
import re

inputStr: str = """0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log25(size)^(1)"""

pattern = re.compile(r"log(?P<base>\d{1,})")

result = pattern.sub(r"1/ln(\g<base>)ln", inputStr)
print(result)

0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * 1/ln(25)ln(size)^(1)


In [32]:
resVar = "Inclusive"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model

['',
 'Loading file: |          | [00:00<?]',
 'Loading file:  67%|██████▋   | [00:00<00:00]',
 'Loading file: 100%|██████████| [00:00<00:00, Creating calltree]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 'Loading file: 100%|██████████| [00:00<00:00, Validating experiment]',
 '',
 'Generating models: |          | [00:00<?]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 'Generating models: 100%|██████████| [00:00<00:00]',
 '\t\tModel: 0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)']

# Extra-P をpythonから実行し、出力を受け取る実験

In [33]:
import subprocess

try:
    res = subprocess.check_output("pwd")
except:
    print("Error.")
print(res)

try:
    res = subprocess.check_output("ls")
except:
    print("Error.")
print(res)


res_str: str = subprocess.run(
    "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
    stdout=subprocess.PIPE,
    text=True,
    shell=True,
).stdout

b'/workspace\n'
b'Dockerfile\nPipfile\nPipfile.lock\nREADME.md\napp\ncsv_files\ndocker-compose.yml\ndocs\nextra-p_docker\nlib\nroot_jupyter\ntesting_pri.ipynb\ntesting_pub.ipynb\ntxt_files\nusedNote.ipynb\nusedNote2.ipynb\nusedNote3.ipynb\nusedNote4.ipynb\nused_scripts\n'


Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]


In [34]:
res_str = res_str.replace("Model: ", "")
print(res_str)

		0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)



In [33]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# Inclusive

In [4]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

trainDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
trainDF_lulesh["InclusivePerCall"] = -1
trainDF_lulesh = trainDF_lulesh.reset_index()
for i, sr in trainDF_lulesh.iterrows():
    trainDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(
        sr["Inclusive"]
    ) / float(sr["#Call"])
trainDF_lulesh = trainDF_lulesh.sort_values("#Call")
# Inclusive列の整形
_before_converted: list[float] = list(trainDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
trainDF_lulesh["Inclusive"] = list(_after_converted)

testDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
testDF_lulesh["InclusivePerCall"] = -1
testDF_lulesh = testDF_lulesh.reset_index()
for i, sr in testDF_lulesh.iterrows():
    testDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(sr["Inclusive"]) / float(
        sr["#Call"]
    )
testDF_lulesh = testDF_lulesh.sort_values("#Call")

# Inclusive列の整形
_before_converted: list[float] = list(testDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
testDF_lulesh["Inclusive"] = list(_after_converted)

functionNames: list[str] = sorted(list(set(trainDF_lulesh["Name"])))
list_series: list[pd.Series] = []
for functionName in functionNames:

    trainDF_perFunc: pd.DataFrame = trainDF_lulesh[
        trainDF_lulesh["Name"] == functionName
    ]
    testDF_perFunc: pd.DataFrame = testDF_lulesh[testDF_lulesh["Name"] == functionName]

    expVar: list[str] = ["process", "iteration", "size"]
    resVar: str

    dict_symbols = {}
    for elem in expVar:
        dict_symbols[elem] = symbols(elem, real=True)
    target_env = [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
        (dict_symbols["process"], test_lulesh_processes[0]),
    ]

    # 総実行時間

    # Extra-Pへの入力ファイルの作榮
    resVar = "Inclusive"
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)

    # Extra-Pの実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)

    model_sympy = sympify(res_str, locals=dict_symbols)
    predicted_all = model_sympy.subs(target_env).evalf()

    # print(predicted_all)

    # 1コール当たりの実行時間

    # Extra-Pへの入力ファイルの作榮
    resVar = "InclusivePerCall"
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)

    # Extra-Pの実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)

    model_sympy = sympify(res_str, locals=dict_symbols)
    predicted_perFunc = model_sympy.subs(target_env).evalf()

    # print(predicted_perFunc)

    # 関数コール回数の予測

    # 関数コール回数予測のためのモデルを構築
    resVar = "#Call"
    result_series_list: list[pd.DataFrame] = []
    list_modelName: list[str] = [
        "modelIp",
        "modelLog",
        "modelLinAndIp",
        "modelLinAndLog",
        "modelIpAndLin",
        "modelIpAndLog",
        "modelLogAndLin",
        "modelLogAndIp",
        "modelProcessDividedByProblemSize",
        "modelProblemSizeDividedByProcess",
        "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
        "modelInfiniteProductOfProblemSizeDividedByProcesses",
        "modelLinearSumOf2elementCombination",
        "modelLinearSumOfElementCombinations",
        "modelLinearSumOf2elementCombinationWithSquared",
        "modelLinearSumOf2elementCombinationWithCubed",
        "modelSquareRootOfProcess",
        "modelSquareRootTimesOtherElems",
        "modelObeyOneParameter",
        "modelLin"
        # "modelBasicTree",
    ]
    trainDF_perFunc = trainDF_perFunc.rename({"Name": "functionName"}, axis="columns")
    testDF_perFunc = testDF_perFunc.rename({"Name": "functionName"}, axis="columns")

    bestModelDict: dict = return_bestModelObject(
        inputDF=trainDF_perFunc,
        list_expVar=expVar,
        list_resVar=[resVar],
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    predicted = float(np.array(bestModel.predict(inputDF=testDF_perFunc[expVar])))
    _call: float = float(testDF_perFunc.iloc[0][resVar])

    # print(_call)

    # 実際の値の取得と結果の整理

    real_time: float = testDF_perFunc.reset_index().loc[0]["Inclusive"]
    real_call: float = testDF_perFunc.reset_index().loc[0]["#Call"]

    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "real_time": real_time,
            "predicted_all": predicted_all,
            "predicted_from_perCall": predicted_perFunc * predicted,
            "real_call": real_call,
            "predicted_call": predicted,
        }
    )

    # print(_series)

    list_series.append(_series)

Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████

In [5]:
pd.concat(list_series, axis=1).T

,functionName,real_time,predicted_all,predicted_from_perCall,real_call,predicted_call
0,.TAU_application,1149.925,1288.20034616662,1288.20034616662,1.0,1.0
1,MPI_Allreduce(),110.861,155.571448846150,184.113257775612,255.0,255.0
2,MPI_Barrier(),0.039,0.00613370466666667,0.00613370466666667,1.0,1.0
3,MPI_Comm_rank(),0.002,0.00340661419639977,0.00249937972674534,2309.0,2309.0
4,MPI_Comm_size(),0.0,4.66073333333333e-7,4.66073333333333e-7,1.0,1.0
5,MPI_Finalize(),0.653,-0.877687749034840,-0.877687749034840,1.0,1.0
6,MPI_Init(),0.593,0.368159207096530,0.368159207096530,1.0,1.0
7,MPI_Irecv(),0.018,0.0237588378914084,0.0173408882000237,8965.8,9907.354054
8,MPI_Isend(),0.248,1.54147758517037,3.31738815983449,8965.8,9907.354054
9,MPI_Reduce(),0.069,0.115818278291975,0.115818278291975,1.0,1.0


In [6]:
testDF_perFunc.reset_index()

,level_0,index,%Time,Exclusive,Inclusive,#Call,#Subrs,functionName,process,iteration,size,InclusivePerCall
0,31,31,0.0,0.000256,2.560000e-07,0.001953,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,512,256,128,0.000131


In [34]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello
